In [ ]:
from model import build_encoder
from c2ae import build_classifier
from classifier_datagenerator import classifier_generator
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K
K.tensorflow_backend._get_available_gpus()

from utils import INPUT_SHAPE, LATENT_DIM, ISIC_TRAIN_SAMPLES,ISIC_VAL_SAMPLES

In [ ]:
MODEL = 'resnet50'

In [ ]:
encoder = build_encoder(LATENT_DIM,MODEL)
classifier = build_classifier(encoder)

In [ ]:
classifier.summary()

In [ ]:
classifier.compile(optimizer=Adam(0.001),loss='categorical_crossentropy',
                   metrics=['categorical_accuracy'])

In [ ]:
batch_size = 1
epochs = 1

In [ ]:
train_data_generator = classifier_generator(batch_size,MODEL,mode='train')
val_data_generator = classifier_generator(batch_size,MODEL,mode='val')

train_nb = int(ISIC_TRAIN_SAMPLES/batch_size)+1
val_nb = int(ISIC_VAL_SAMPLES/batch_size)+1

In [ ]:
filepath='models/'+MODEL+"-save-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', 
                             save_weights_only=False,
                             verbose=1, save_best_only=True, mode='min',
                             period=1)
callbacks_list = [checkpoint]

In [ ]:
train_history = classifier.fit_generator(train_data_generator,
                                         steps_per_epoch=train_nb,
                                         epochs=epochs,
                                         validation_data = val_data_generator,
                                         validation_steps = val_nb,
                                         callbacks=callbacks_list)